In [1]:
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.collections import LineCollection
from matplotlib import cm
import numpy as np
import pandas as pd

In [2]:
#setup plotting 

plotting.setup_mpl()

# Enable the cache
ff1.Cache.enable_cache('cache') 

# Get rid of some pandas warnings that are not relevant for us at the moment
pd.options.mode.chained_assignment = None

In [3]:
import json
with open('track.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data['Tracks'])


In [18]:
# Load the session data
hh=[]
for i in range(len(df)):
    
    track = df.Name[i]

    ses = ff1.get_session(2024, track, 'R')


    # Get the laps
    ses.load()
    #laps = ses.laps()
    circuit_info = ses.get_circuit_info()
    g= df.corners[i]
    h= df.overtake[i]
    corner_dist = circuit_info.corners[circuit_info.corners.Number.isin(g)].Distance.to_list()
    overtake_dist = circuit_info.corners[circuit_info.corners.Number.isin(h)].Distance.to_list()


    laps = ses.laps
    lap = laps.pick_driver('HAM').iloc[2]

    # Get telemetry data for this lap
    telemetry = lap.get_telemetry()
        

        
    





core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.9]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Spanish Grand Prix - Rac

In [19]:
hh

[41, 163, 472, 53, 'error', 101]

In [5]:
#ses.load()  # Load the session data

# Get circuit information
circuit_info = ses.get_circuit_info()
g=[1,2,3,4]
bahrain = circuit_info.corners[circuit_info.corners.Number.isin(g)].Distance
bahrain


0     343.852594
1     425.515181
2    1060.244502
3    1212.127634
Name: Distance, dtype: float64

In [6]:
laps = ses.laps 

In [27]:
lap = laps.pick_driver('GAS').iloc[8]

# Get telemetry data for this lap
telemetry = lap.get_telemetry()

# Find index before braking zone
cornerindex = telemetry[(telemetry.Distance > (bahrain[0]-5)) & (telemetry.Distance < (bahrain[0]+5))].index[0] 

last_acc_index = telemetry.iloc[cornerindex-35:cornerindex]['Throttle'][::-1].loc[lambda x:x==100].index[4]

Pre_Brake_Dist = telemetry.iloc[last_acc_index]['Distance']

Pre_Brake_Speed = telemetry.iloc[last_acc_index]['Speed']

corner_speed = telemetry.iloc[cornerindex]['Speed']


In [28]:
print(Pre_Brake_Speed)
print(corner_speed)

311
150


In [29]:
lap_tyre_df = laps[['Driver', 'LapTime','Compound', 'TyreLife']]
lap_driver = lap_tyre_df.groupby(['Driver','Compound'], as_index=False)['TyreLife'].max()
Comp_av = lap_driver.groupby(['Compound'],as_index=False)['TyreLife'].mean()


In [31]:
Comp_av

,Compound,TyreLife
0,HARD,26.0000
1,MEDIUM,11.4375
2,SOFT,6.0000
